# Wikipedia Notable Life Expectancies
# [Notebook 6 : Data Cleaning Part 5](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean5_thanak_2022_07_17.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [3]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [4]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean4.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean4", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98060 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [5]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98058,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,journalist,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,0,0,0,1,0,0,0,1
98059,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [6]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
38182,17,Ben H. Procter,", 85, American author and football player , Parkinson's disease.",https://en.wikipedia.org/wiki/Ben_H._Procter,8,2012,April,Rams,None,author,Parkinson disease,None,None,None,None,None,None,None,None,85.0,None,United States of America,United States of America,Rams,2.197225,0,0,0,0,0,0,1,0,0,0,0,0,1
33812,14,Sean Boru,", 57, Irish actor and author.",https://en.wikipedia.org/wiki/Sean_Boru,4,2011,February,None,None,,None,None,None,None,None,None,None,None,None,57.0,None,Ireland,None,None,1.609438,0,0,0,0,0,1,0,0,0,0,0,0,1
52258,2,Leslie Chamberlain,", 81, English rugby league player.",https://en.wikipedia.org/wiki/Leslie_Chamberlain,7,2015,March,None,None,rugby league player,None,None,None,None,None,None,None,None,None,81.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,2.079442,0,0,0,0,0,0,0,0,0,0,0,0,0
27782,28,Manila,", 26, American Thoroughbred racehorse, aortic ring rupture.",https://en.wikipedia.org/wiki/Manila_(horse),5,2009,February,None,None,Thoroughbred racehorse,aortic ring rupture,None,None,None,None,None,None,None,None,26.0,None,United States of America,None,None,1.791759,0,0,0,0,0,0,0,0,0,0,0,0,0
90311,26,H. S. Doreswamy,", 103, Indian journalist and independence activist, cardiac arrest.",https://en.wikipedia.org/wiki/H._S._Doreswamy,25,2021,May,None,None,journalist and independence activist,cardiac arrest,None,None,None,None,None,None,None,None,103.0,None,India,None,None,3.258097,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [7]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98060 entries, 0 to 98059
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98060 non-null  object 
 1   name                       98060 non-null  object 
 2   info                       98060 non-null  object 
 3   link                       98060 non-null  object 
 4   num_references             98060 non-null  int64  
 5   year                       98060 non-null  int64  
 6   month                      98060 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98028 non-null  object 
 10  info_3                     48895 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting `known_for` values by creating `known_for_dict_7`

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [8]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [10]:
# Code to check each value
roles_list.pop()

'writer'

<IPython.core.display.Javascript object>

In [442]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [index for index in df.index if "writer" in df.loc[index, "info"]], "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [443]:
specific_roles_list.index("society hostess and writer")

1006

<IPython.core.display.Javascript object>

In [444]:
specific_roles_list = specific_roles_list[:1007]

<IPython.core.display.Javascript object>

In [616]:
# Code to check each specific value
specific_roles_list.pop()

'TV writer and producer'

<IPython.core.display.Javascript object>

In [562]:
# Example code to quick-screen values that may overlap categories
df.loc[[index for index in df.index if "counterculture" in df.loc[index, "info"]]]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
1385,28,Jerry Rubin,", 56, American social activist, anti-war leader, and counterculture icon, traffic accident.",https://en.wikipedia.org/wiki/Jerry_Rubin,49,1994,November,None,None,social activist,anti war leader,and counterculture icon,traffic accident,None,None,None,None,None,None,56.0,None,United States of America,None,None,3.912023,0,0,0,0,0,0,0,0,0,0,0,0,0
8495,28,Kerry Wendell Thornley,", 60, American counterculture figure and writer, cardiac arrest.",https://en.wikipedia.org/wiki/Kerry_Wendell_Thornley,20,1998,November,None,None,counterculture figure and writer,cardiac arrest,None,None,None,None,None,None,None,None,60.0,None,United States of America,None,None,3.044522,0,0,0,0,0,0,0,0,0,0,0,0,0
13400,6,Jacques Katmor,", 63, Israeli counterculture experimental filmmaker and artist, alcoholism.",https://en.wikipedia.org/wiki/Jacques_Katmor,24,2001,September,None,None,counterculture experimental filmmaker and artist,alcoholism,None,None,None,None,None,None,None,None,63.0,None,Israel,None,None,3.218876,0,0,0,0,0,0,0,0,0,0,0,0,0
13527,30,John C. Lilly,", 86, American physician, writer, inventor and counterculture scientist.",https://en.wikipedia.org/wiki/John_C._Lilly,44,2001,September,None,None,physician,writer,inventor and counterculture scientist,None,None,None,None,None,None,None,86.0,None,United States of America,None,None,3.806662,0,0,0,0,0,0,0,0,0,0,0,0,0
33683,30,Brian Barritt,", 76, English author, artist, and counterculture figure.",https://en.wikipedia.org/wiki/Brian_Barritt,8,2011,January,None,None,author,artist,and counterculture figure,None,None,None,None,None,None,None,76.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,2.197225,0,0,0,0,0,0,0,0,0,0,0,0,0
48760,1,Stephen Gaskin,", 79, American counterculture figure, peace activist and commune founder .",https://en.wikipedia.org/wiki/Stephen_Gaskin,8,2014,July,The Farm,None,counterculture figure,peace activist and commune founder,None,None,None,None,None,None,None,None,79.0,None,United States of America,None,The Farm,2.197225,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [561]:
# Example code to quick-screen values that may overlap categories
df.loc[
    [
        index
        for index in df.index
        if "counterculture figure and writer" in df.loc[index, "info"]
    ]
]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
8495,28,Kerry Wendell Thornley,", 60, American counterculture figure and writer, cardiac arrest.",https://en.wikipedia.org/wiki/Kerry_Wendell_Thornley,20,1998,November,None,None,counterculture figure and writer,cardiac arrest,None,None,None,None,None,None,None,None,60.0,None,United States of America,None,None,3.044522,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [477]:
# Example code to quick-check a specific entry
df[df["info_2"] == "singer songwriter and guitarist of Big Country and The Raphaels"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
13955,16,Stuart Adamson,", 43, Scottish singer-songwriter and guitarist of Big Country and The Raphaels.",https://en.wikipedia.org/wiki/Stuart_Adamson,25,2001,December,None,None,singer songwriter and guitarist of Big Country and The Raphaels,None,None,None,None,None,None,None,None,None,43.0,None,Scotland,None,None,3.258097,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = [
    'former executive editor of Richard Nixon and speech',
    'and free speech advocate',
    'freedom of speech advocate',
    'lawyer and free speech activist',
    'civil rights activist and speech',
    'and speech',
    'political speech',
    'and free speech activist',
    'speech',

    
]

arts = [
    'rock and roll singer songwriter',
    'writer and radio dramatist',
    'writer of Berserker series',
    'writer and fashion editor',
    'comedy writer and lyricist',
    'writer and novelist in the Kannada language',
    'television writer for and',
    'writer of books and modules',
    'science fiction and western writer',
    'folk blues singer songwriter',
    'songwriter and columnist',
    'children writer and comic strip cartoonist',
    'spy fiction writer',
    'Emmy winning television comedy writer',
    'political journalist and food writer',
    'writer of all episodes of the long running radio serial',
    'writer of descent',
    'artist and screenwriter',
    'wine writer',
    'thriller writer and James Bond continuation novelist',
    'jazz writer',
    'writer and beer expert',
    'jazz musician and comedy writer',
    'television writer blacklisted by the House Un Activities Committee',
    'Motown songwriter and record producer',
    'writer and radio producer',
    'comic book writer and editor for DC Comics',
    'Chukchi language writer',
    'writer and script editor',
    'Christian rock singer songwriter',
    'writer of detective fiction',
    'novelist and non fiction writer',
    'cookbook writer',
    'playwright and scriptwriter',
    'motion picture writer director',
    'Broadway producer and writer',
    'film and television producer and screenwriter',
    'vocalist and songwriter',
    'country music manager and songwriter',
    'rock musician and songwriter',
    'classical guitarist and writer',
    'techno singer songwriter',
    'songwriter and drummer',
    'travel writer and broadcaster',
    'writer and editor',
    'writer and national artist',
    'Emmy nominated television writer',
    'nonfiction writer',
    'copywriter and executive',
    'comics writer and newspaper editor',
    'film director and script writer',
    'literary agent and writer',
    'writer of screenplays',
    'Braziliam writer and journalist',
    'co writer of the s national anthem',
    'songwriter and folksinger',
    'writer and radio broadcaster',
    'writer and journalist on',
    'novelist and soap opera writer',
    'country western songwriter',
    'libertarian writer and',
    'TV writer',
    'gardening writer',
    'architectural critic and writer',
    'animation writer producer',
    'and chess writer',
    'producer and writer of TV comedies',
    'lead singer songwriter musician of rock band God Lives Underwater',
    'resident writer and film critic',
    'writer and puppeteer for',
    'long time Associated Press cuisine writer',
    '"New Wave" vallenato singer and songwriter',
    'jazz pianist and film music writer',
    'R&B songwriter and record producer',
    'dramatist and writer',
    'writer editor publisher',
    'writer and Nobel Prize winner',
    'mystery novel writer',
    'Cree writer',
    'film producer and writer',
    'writer and architectural conservationist',
    'pop singer and writer',
    'poet writer',
    'writer and art critic',
    'experimental writer',
    'writer and wargame designer',
    'writer and news presenter',
    'songwriter and harpist',
    'writer and sound',
    'writer and music journalist',
    'potter and writer',
    'silent film screenwriter',
    'illustrator and writer',
    'sitcom writer',
    'percussionist and songwriter',
    'rock keyboardist and songwriter',
    'writer and speaker',
    'rhythm and blues and jazz musician and songwriter',
    'singer songwriter and writer',
    'radio writer',
    'writer and television director',
    'writer and television personality',
    'and technical writer',
    'television writer and director',
    'short story and travel writer',
    'Motown songwriter',
    'psychedelic rock songwriter and musician',
    'Lakota writer and',
    'Romani writer',
    'BBC Radio writer',
    'manga artist and writer',
    'musician and score writer',
    'game show writer',
    'film producer and television screenwriter',
    'poet and writer in Gujarati language',
    'comedian and television writer',
    'writer and book editor',
    'television comedy writer and producer',
    'comic book writer and music critic',
    'writer and magician',
    'anime writer and producer',
    'writer and lyricist',
    'detective writer',
    'television and radio writer',
    'fiddler and singer songwriter',
    'Emmy Award winning screenwriter',
    'cook and food writer',
    'writer and investigative journalist',
    'singer songwriter and dancer',
    'songwriter and guitarist',
    'Academy Award nominated playwright and screenwriter',
    'First Nations singer songwriter',
    'cookery and travel writer',
    'songwriter and radio talk show host',
    'television host and writer',
    'Emmy Award winning television writer',
    'rockabilly musician and songwriter',
    'film and TV producer and writer',
    'Grammy winning songwriter',
    'jazz organist and songwriter',
    'copywriter and product spokesperson',
    'writer and newspaper editor',
    'speaker and writer',
    'painter and television screenplay writer',
    'comic strip writer and editor',
    'screenplay writer',
    'writer and editor of science fiction and fantasy',
    'writer and television journalist',
    'TV producer and writer',
    'Academy Award nominated screenwriter',
    'script writer and producer',
    'Urdu language writer',
    "musician and songwriter for Guns N' Roses",
    'songwriter and music promoter',
    'mystery writer and novelist',
    'travel writer and',
    'writer and small press printer',
    'writer and theatre critic',
    'and poet and screenwriter',
    'writer and music executive',
    'playwright and writer',
    'poet and prose writer',
    'songwriter and television producer',
    'author and ghostwriter',
    'folksinger and songwriter',
    'writer under the pseudonym',
    'comic books illustrator and writer',
    'writer and Pulitzer Prize winner',
    'folk music songwriter and musician',
    'television scriptwriter',
    'songwriter and music manager',
    'writer and surrealist poet',
    'R&B and blues singer and songwriter',
    'short story writer and scriptwriter',
    'writer and Nobel Prize laureate',
    'comedy writer and radio and television personality',
    'writer and expert on country houses',
    'Pulitzer Prize winning war correspondent and writer',
    'writer and science fiction author',
    'novelist and a mystery writer',
    'Western writer',
    'writer and raconteur',
    'tenor and songwriter',
    'singer poet and songwriter',
    'folk music singer songwriter',
    'writer of Westerns',
    'ballet and dance writer',
    'author and the wife of writer Dylan Thomas',
    'literary critic and writer',
    'and film director and screenwriter',
    'writer and humorist',
    'composer and writer on music',
    'lyricist and songwriter',
    'writer and author',
    'rockabilly and country singer and songwriter',
    'writer and only child author Jack Kerouac and Joan Haverty Kerouac',
    'Poet; writer',
    'humorist and writer',
    'Urdu poet and screenwriter',
    'director and scriptwriter',
    'playwright and screenwriter specialising in comedies',
    'writer of crime fiction',
    'film critic and travel writer',
    'writer who won the Newbery Medal',
    'writer of short stories and poetry',
    'editor and writer on music and dance',
    'songwriter and impresario',
    'journalist and writer on music and',
    'writer and film',
    'songwriter and session musician',
    'screenwriter who won the Academy Award for Best Story for',
    'and writer on the language',
    'gospel bass singer and songwriter',
    'folk rock singer songwriter and film producer',
    'producer and film and television writer',
    'author and screenwriter',
    'screenwriter and World War II foreign correspondent',
    'writer of fiction and screenplays',
    'songwriter and jazz pianist',
    'architectural photographer and writer',
    'writer of literature',
    '; also a world renowned poet and writer',
    'singer songwriter and guitarist of Big Country and The Raphaels',
    'comic strip and comic book writer and cartoonist',
    'Sami writer',
    'country and rockabilly singer',
    'born singer songwriter',
    'country music songwriter and record producer',
    'writer of the Old West',
    'Arabian writer of Arab descent',
    'rockabilly songwriter and singer',
    'and writer on Old Testament issues',
    'Blackfeet and Gros Ventre writer and poet',
    'newspaper reporter and writer',
    'comic book writer artist',
    'R&B musician songwriter',
    'agent for writers and cartoonists',
    'writer of stories and news articles',
    'singer songwriter and playwright',
    'writer of more than children books',
    'classical music broadcaster and writer',
    'entertainer and songwriter',
    'writer and illustrator of children books',
    'bebop and jazz pianist and songwriter',
    'television and film producer and screenwriter',
    'popular singer and songwriter',
    'screenwriter and producer for television',
    'Creole zydeco musician and songwriter',
    'writer of drama and short stories',
    'juvenile literature writer',
    'short story writer',
    'novelist and writer for children',
    'writer of comic books and strips',
    'editorial writer',
    'rhythm and blues and jazz singer and songwriter',
    'R&B musician and songwriter',
    'writer and director of plays and films',
    'short story writer and essayist',
    'jazz writer and record producer',
    'writer also known as ""',
    'reggae singer songwriter',
    'screenwriter and novelist',
    'and freethought writer',
    'comic book writer for DC Comics',
    'writer and movie director',
    'writer of pulp fiction',
    'media personality and writer',
    'songwriter and bass guitarist',
    'jazz and calypso musician and songwriter',
    'fiction writer and essayist',
    'poet and short story writer',
    'screenwriter and filmmaker',
    'novelist and story writer',
    'children author and writer',
    'comic strip writer and artist',
    'author and screenplay writer',
    'novelist and radio script writer',
    'writer from Nadu',
    'writer and music producer',
    'rock and roll singer and songwriter',
    'telenovela screenwriter',
    'comedy writer and theater owner',
    'cartoonist and comics writer and artist',
    'food writer and broadcaster',
    'documentary film director and screenwriter',
    'cartoonist and comic book writer',
    'journalist and culture writer',
    'rock singer and songwriter',
    'TV writer and producer'
    
    
    
    
    
    
    
    'writer and journalist',
    'and hymn writer',
    'writer in',
    'of and writer',
    'and science fiction writer',
    'based writer',
    'and crime writer',
    'and non fiction writer',
    'Arabian writer and',
    'children writer and',
    'and food writer',
    'singer songwriter and',
    'jazz writer and',
    'science fiction writer and',
    'non fiction writer and',
    'fiction writer and',
    'food writer and',
    'and screenwriter and playwright',
    'head writer for',
    'and screenwriter',
    'and science writer',
    'and technology writer',
    'television writer for',
    'songwriter and',
    'and writer',
    'writer and a',
    'writer and',
    'writer on'
]
sports = [
    'hiking',  # after arts

]
sciences = [
    'manual typewriter expert', # before arts
    'speech therapist', # before politics_govt_law
    'speech pathologist',
    'speech recognition researcher'
]

business_farming = [
]
academia_humanities = [
    'Yiddish language preservationist',
]
law_enf_military_operator = []
spiritual = []
social = [
    'society hostess'
]
crime = []
event_record_other = []
other_species = [
]

#### Creating `known_for_dict_1` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict_1 = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "arts": arts,
    "sports": sports,
    "law_enf_military_operator": law_enf_military_operator,
    "politics_govt_law": politics_govt_law,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict_1

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
df["num_categories"] = df[known_for_dict_1.keys()].sum(axis=1)

print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

In [ ]:
#### Observations:
- We will proceed to build `known_for_dict_6` for the next iteration.